In [24]:
import autograd.numpy as np

layers_sizes = [100,25,4]

n_hidden = np.size(layers_sizes)
print(n_hidden)

P = [None]*(n_hidden + 1)
print(len(P))
 
P[0] = np.random.randn(layers_sizes[0], 2 + 1 ) # 2 since we have two points, +1 to include bias
#print(P[0])
for l in range(1,n_hidden):
    P[l] = np.random.randn(layers_sizes[l], layers_sizes[l-1] + 1) # +1 to include bias

P[-1] = np.random.randn(1,layers_sizes[-1]+1)
#print(P[-1])

for i in range(len(P)):
    print(P[i].shape)

3
4
(100, 3)
(25, 101)
(4, 26)
(1, 5)


In [30]:
num_points = 1; x_prev = [[4],[3]]

x_prev = np.concatenate((np.ones((1,num_points)), x_prev ), axis = 0)
print(x_prev)

a = np.linspace(0,1,2)
print(a)
print(a[:-1])

[[1.]
 [4.]
 [3.]]
[0. 1.]
[0.]
